<a href="https://colab.research.google.com/github/R-aryan/Name_Entity_Recognition/blob/master/NER_Medical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import unicodedata
 
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense
from keras.layers import TimeDistributed, Dropout, Bidirectional
 
# Defining Constants
 
# Maximum length of text sentences
MAXLEN = 180
# Number of LSTM units
LSTM_N = 150
# batch size
BS=48

In [0]:
data = pd.read_csv("/content/drive/My Drive/NER_Disease/train.csv", encoding="latin1")
data.head()

,id,Doc_ID,Sent_ID,Word,tag
0,1,1,1,Obesity,O
1,2,1,1,in,O
2,3,1,1,Low-,O
3,4,1,1,and,O
4,5,1,1,Middle-Income,O


In [0]:
test_data = pd.read_csv("/content/drive/My Drive/NER_Disease/test.csv", encoding="latin1")
test_data.head()

,id,Doc_ID,Sent_ID,Word
0,4543834,30001,191283,CCCVA
1,4543835,30001,191283,","
2,4543836,30001,191283,MANOVA
3,4543837,30001,191283,","
4,4543838,30001,191283,my


In [0]:
print(data['tag'].unique())

['O' 'B-indications' 'I-indications']


Creating Word & Tag dictionary

In [0]:
print("Number of uniques docs, sentences and words in Training set:\n",data.nunique())
print("\nNumber of uniques docs, sentences and words in Test set:\n",test_data.nunique())
 
# Creating a vocabulary
words = list(set(data["Word"].append(test_data["Word"]).values))
words.append("ENDPAD")
 
# Converting greek characters to ASCII characters eg. 'naïve café' to 'naive cafe'
words = [unicodedata.normalize('NFKD', str(w)).encode('ascii','ignore') for w in words]
n_words = len(words)
print("\nLength of vocabulary = ",n_words)
 
tags = list(set(data["tag"].values))
n_tags = len(tags)
print("\nnumber of tags = ",n_tags)
 
# Creating words to indices dictionary.
word2idx = {w: i for i, w in enumerate(words)}
# Creating tags to indices dictionary.
tag2idx = {t: i for i, t in enumerate(tags)}

Number of uniques docs, sentences and words in Training set:
 id         4543833
Doc_ID       30000
Sent_ID     191282
Word        184505
tag              3
dtype: int64

Number of uniques docs, sentences and words in Test set:
 id         2994463
Doc_ID       20000
Sent_ID     125840
Word        139891
dtype: int64

Length of vocabulary =  257203

number of tags =  3


 Getting Train & Test Sentences

In [0]:
def get_tagged_sentences(data):


    agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(), s["tag"].values.tolist())]
    grouped = data.groupby("Sent_ID").apply(agg_func)
    sentences = [s for s in grouped]
    return sentences


def get_test_sentences(data):

    agg_func = lambda s: [w for w in s["Word"].values.tolist()]
    grouped = data.groupby("Sent_ID").apply(agg_func)
    sentences = [s for s in grouped]
    return sentences

In [0]:
# Getting training sentences in a list
sentences = get_tagged_sentences(data)
print("First 2 sentences in a word list format:\n",sentences[0:2])


First 2 sentences in a word list format:
 [[('Obesity', 'O'), ('in', 'O'), ('Low-', 'O'), ('and', 'O'), ('Middle-Income', 'O'), ('Countries', 'O'), (':', 'O'), ('Burden', 'O'), (',', 'O'), ('Drivers', 'O'), (',', 'O'), ('and', 'O'), ('Emerging', 'O'), ('Challenges', 'O'), ('.', 'O')], [('We', 'O'), ('have', 'O'), ('reviewed', 'O'), ('the', 'O'), ('distinctive', 'O'), ('features', 'O'), ('of', 'O'), ('excess', 'O'), ('weight', 'O'), (',', 'O'), ('its', 'O'), ('causes', 'O'), (',', 'O'), ('and', 'O'), ('related', 'O'), ('prevention', 'O'), ('and', 'O'), ('management', 'O'), ('efforts', 'O'), (',', 'O'), ('as', 'O'), ('well', 'O'), ('as', 'O'), ('data', 'O'), ('gaps', 'O'), ('and', 'O'), ('recommendations', 'O'), ('for', 'O'), ('future', 'O'), ('research', 'O'), ('in', 'O'), ('low-', 'O'), ('and', 'O'), ('middle-income', 'O'), ('countries', 'O'), ('(', 'O'), ('LMICs', 'O'), (')', 'O'), ('.', 'O')]]


In [0]:
# Getting test sentences in a list
test_sentences = get_test_sentences(test_data)
print("First 2 sentences in a word list format:\n",test_sentences[0:2])
print(type(test_sentences))


First 2 sentences in a word list format:
 [['CCCVA', ',', 'MANOVA', ',', 'my', 'black', 'hen', '.'], ['Comments', 'on', 'repeated', 'measures', '.']]
<class 'list'>


In [0]:
X_test = [[word2idx[unicodedata.normalize('NFKD', str(w)).
encode('ascii','ignore')] for w in s] for s in test_sentences]

print(words[1:10])

print(type(X_test))

[b'VPDs', b'0.14-0.87', b'facilitated-diffusion', b'Powassan', b'alpha-monoclonal', b'HHHFNCT', b'decentralize', b'1A135', b'plasmid-derived']
<class 'list'>


Feature Extraction for DL Model

In [0]:
# Converting words to indices for test sentences (Features)
# Converting greek characters to ASCII characters in train set eg. 'naïve café' to 'naive cafe'
X = [[word2idx[unicodedata.normalize('NFKD', str(w[0])).
encode('ascii','ignore')] for w in s] for s in sentences]
 
# Converting words to indices for test sentences (Features)
# Converting greek characters to ASCII characters in test-set eg. 'naïve café' to 'naive cafe'
X_test = [[word2idx[unicodedata.normalize('NFKD', str(w)).
encode('ascii','ignore')] for w in s] for s in test_sentences]
 
'''
Padding train and test sentences to 180 words.
Sentences of length greater than 180 words are truncated.
Sentences of length less than 180 words are padded with a high value.
'''
X = pad_sequences(maxlen=MAXLEN, sequences=X, padding="post", value=n_words - 1)
X_test = pad_sequences(maxlen=MAXLEN, sequences=X_test, padding="post", value=n_words - 1)
 
# Converting tags to indices for test sentences (labels)
y = [[tag2idx[w[1]] for w in s] for s in sentences]
# Padding tag labels to 180 words.
y = pad_sequences(maxlen=MAXLEN, sequences=y, padding="post", value=tag2idx["O"])
 
# Making labels in one hot encoded form for DL model
y = [to_categorical(i, num_classes=n_tags) for i in y]

Building  Model

In [0]:
# 180 dimensional word indices as input
input = Input(shape=(MAXLEN,))
 
# Embedding layer of same length output (180 dim embedding will be generated)
model = Embedding(input_dim=n_words, output_dim=MAXLEN, input_length=MAXLEN)(input)
 
# Adding dropout layer
model = Dropout(0.2)(model)
 
# Bidirectional LSTM to learn from both forward as well as backward context
model = Bidirectional(LSTM(units=LSTM_N, return_sequences=True, recurrent_dropout=0.1))(model)
 
# Adding a TimeDistributedDense, to applying a Dense layer on each 180 timesteps
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model) # softmax output layer
model = Model(input, out)
 
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X, np.array(y), batch_size=BS, epochs=2, validation_split=0.05, verbose=1)

model.save('/content/drive/My Drive/NER_Disease/my_NER_model.h5')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 181717 samples, validate on 9565 samples
Epoch 1/2





181717/181717 [==============================] - 2320s 13ms/step - loss: 0.0073 - acc: 0.9981 - val_loss: 0.0041 - val_acc: 0.9988
Epoch 2/2
181717/181717 [==============================] - 2304s 13ms/step - loss: 0.0029 - acc: 0.9990 - val_loss: 0.0038 - val_acc: 0.9989


In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 180)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 180, 180)          46296540  
_________________________________________________________________
dropout_1 (Dropout)          (None, 180, 180)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 180, 300)          397200    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 180, 3)            903       
Total params: 46,694,643
Trainable params: 46,694,643
Non-trainable params: 0
_________________________________________________________________


Prediction on Test Set

In [0]:
del model

In [0]:
model = load_model('/content/drive/My Drive/NER_Disease/my_model.h5')

In [0]:
# Predicting on trained model
pred = model.predict(X_test)
print("Predicted Probabilities on Test Set:\n",pred.shape)
# taking tag class with maximum probability
pred_index = np.argmax(pred, axis=-1)
print("Predicted tag indices: \n",pred_index.shape)

Predicted Probabilities on Test Set:
 (125840, 180, 3)
Predicted tag indices: 
 (125840, 180)


In [0]:
# Flatten both the features and predicted tags
ids,tagids = X_test.flatten().tolist(), pred_index.flatten().tolist()
 
# converting each word indices back to words
words_test = [words[ind].decode('utf-8') for ind in ids]
# converting each predicted tag indices back to tags
tags_test = [tags[ind] for ind in tagids]
print("Length of words in Padded test set:",len(words_test))
print("Length of tags in Padded test set:",len(tags_test))
print("\nCheck few of words and predicted tags:\n",words_test[:10],tags_test[:10])


Length of words in Padded test set: 22651200
Length of tags in Padded test set: 22651200

Check few of words and predicted tags:
 ['CCCVA', ',', 'MANOVA', ',', 'my', 'black', 'hen', '.', 'ENDPAD', 'ENDPAD'] ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [0]:
from keras.models import load_model

model.save('/content/drive/My Drive/NER_Disease/my_model.h5')

In [0]:
'''
The task here is to convert padded fixed 180 dimensional predicted tags
to variable length test set sentences.
1. If the sentences have word length shorter than 180,
   then predcited tags are skipped.
2. If the sentences have word length longer than 180,
   then all extra words are tagged with "O" tag class.
'''
 
i=0
j=1
predicted_tags = []
predicted_words=[]
counts = test_data.groupby('Sent_ID')['id'].count().tolist()
 
for index,count in enumerate(counts):
    if count <= MAXLEN:
        predicted_tags.append(tags_test[i:i+count])
        predicted_words.append(words_test[i:i+count])
    else:
        predicted_tags.append(tags_test[i:i+MAXLEN])
        predicted_words.append(words_test[i:i+MAXLEN])
        out = ['O']*(count-MAXLEN)
        predicted_tags.append(out)
        predicted_words.append(out)
 
    i=j*MAXLEN
    j=j+1
 
predictions_final = [item for sublist in predicted_tags for item in sublist]
words_final = [item for sublist in predicted_words for item in sublist]
print("\nLength of test set words and predicted tags should match.")
print("Length of predicted tags:",len(predictions_final))
print("Length of words in test set:",test_data['Word'].size)


Length of test set words and predicted tags should match.
Length of predicted tags: 2994463
Length of words in test set: 2994463


In [0]:
#df = pd.read_csv("/content/drive/My Drive/NER_Disease/sample_submission.csv", encoding="latin1")
# Creating a dataframe in the submission format
df_results = pd.DataFrame({'word':words_final,'tag':predictions_final})
# writing csv submission file
df_results.to_csv('/content/drive/My Drive/NER_Disease/prediction_final.csv',sep=",", index=None)
df_results.head(17)

,word,tag
0,CCCVA,O
1,",",O
2,MANOVA,O
3,",",O
4,my,O
5,black,O
6,hen,O
7,.,O
8,Comments,O
9,on,O


In [0]:
# def predict_output(X_test,words):


  

#   # Predicting on trained model
#   pred = model.predict(X_test)
#   print("Predicted Probabilities on Test Set:\n",pred.shape)
#   # taking tag class with maximum probability
#   pred_index = np.argmax(pred, axis=-1)
#   print("Predicted tag indices: \n",pred_index.shape)


#   # Flatten both the features and predicted tags
#   ids,tagids = X_test.flatten().tolist(), pred_index.flatten().tolist()
 
#   # converting each word indices back to words
#   words_test = [words[ind].decode('utf-8') for ind in ids]
#   # converting each predicted tag indices back to tags
#   tags_test = [tags[ind] for ind in tagids]
#   #print(tagids)
#   print("Length of words in Padded test set:",len(words_test))
#   print("Length of tags in Padded test set:",len(tags_test))
#   print("\nCheck few of words and predicted tags:\n",words_test[:10],tags_test[:10])

In [0]:
# def convert_input(str_test):


#   # doc = nlp(str_test)
#   # print(doc.ents)
 

#   str_test = list(str_test.split())
 

#   words= list(set(str_test))

#   words.append("ENDPAD")

#   n_words = len(words)

#   words= [unicodedata.normalize('NFKD', str(w)).encode('ascii','ignore') for w in words]

#   # Creating words to indices dictionary.
#   word2idx = {w: i for i, w in enumerate(words)}

#   X_test = [[word2idx[unicodedata.normalize('NFKD', str(w)).encode('ascii','ignore')] for w in str_test]]

#   X_test = pad_sequences(maxlen=MAXLEN, sequences=X_test, padding="post", value=n_words - 1)

#   predict_output(X_test,words)